In [1]:
# 先从简单的CNN开始写吧，嘶，主要是数据太少了orzzzzz
# 所以只能做分类了，完蛋，我开始写之前怎么就没想到这点呢
# 没事，3个类，33个数据一类，勉勉强强够了，大不了我自己再洗一年的数据

import pandas as pd
from utils import randomSplit, Accumulator
from torch import nn
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler  # 归一化
from sklearn.model_selection import KFold         # k折交叉验
from sklearn.decomposition import PCA             # pca

def rank2class(df):
    if df['LgRk'] <= 6:
        return 0
    elif 6 < df['LgRk'] <= 13:
        return 1
    else:
        return 2

In [2]:
class myDataset():
    def __init__(self, renorm=False, pca=False):
        self.player = pd.read_csv('datacleaning/modified_data.csv', header=0, encoding='unicode_escape', delimiter=';')
        self.player = self.player.drop(['Rk', 'Player', 'Nation', 'Pos', 'Comp', 'Age', 'Born'], axis=1)
        if renorm:
            scaler = StandardScaler()  # z score 归一化
            for col in self.player.columns:
                if col == 'Squad': continue
                self.player[[col]] = scaler.fit_transform(self.player[[col]])


        self.league = pd.read_excel('datacleaning/Big 5 European Leagues.xlsx', sheet_name='Big 5 European Leagues Stats')
        self.league['tier'] = self.league.apply(rank2class, axis=1)
        self.league = self.league[['Squad', 'tier']]

        self.X = []
        self.y = []
        for id in self.league.index:
            squad = self.league.at[id, 'Squad']
            self.X.append(self.Squad2Player(squad).reshape((1, -1)))  # reshape (207,)  -> (1, 270)
            self.y.append(self.league.at[id, 'tier'])

        self.X = np.concatenate(self.X, axis=0)
        self.y = np.array(self.y)

        if pca:
            pca = PCA(n_components=15)   # 15 is best
            pca.fit(self.X)
            self.X = pca.transform(self.X)
            #print(sum(pca.explained_variance_ratio_))  # 测试此时的总解释方差达到 99%

    def Squad2Player(self, squad):
        "输入squad的名字，输出球队球员的数据"
        re = self.player[self.player['Squad'] == squad].drop('Squad', axis=1)
        re = re.values
        mean = np.mean(re, axis=0)
        max = np.max(re, axis=0)
        min = np.min(re, axis=0)
        re = np.concatenate((mean, max, min), axis=0)
        #print(re.shape) # 135*5
        return re      

    # 使用K折交叉验证
    def DataIter(self, K=10, shuffle=True):
        KF = KFold(n_splits=K, shuffle=shuffle)  
        for train_index, test_index in KF.split(self.X):
            #print("TRAIN", train_index, "TEST", test_index)
            X_train, X_test = self.X[train_index], self.X[test_index]
            Y_train, Y_test = self.y[train_index], self.y[test_index]
            yield X_train, Y_train, X_test, Y_test
    

mydataset = myDataset()
print(mydataset.X.shape)

(98, 405)


In [3]:
# SVM
from sklearn import svm
mydataset = myDataset(renorm=0)
record = []
for X_train, Y_train, X_test, Y_test in mydataset.DataIter():
    model = svm.SVC(
        C=1.0,
        kernel='linear',  # linear效果最好
        gamma='scale',
        decision_function_shape='ovo'
    )
    model.fit(X_train, Y_train.ravel())    #ravel函数在降维时默认是行序优先
    #利用classifier.score（）分别计算训练集和测试集的准确率。
    train_score = model.score(X_train, Y_train)
    print("训练集：",train_score, end='  ')
    test_score = model.score(X_test, Y_test)
    print("测试集：",test_score)
    record.append(test_score)
    
print(f'估计泛化误差：{sum(record)/len(record):.3f}')


训练集： 1.0  测试集： 0.7
训练集： 1.0  测试集： 0.7
训练集： 1.0  测试集： 0.5
训练集： 1.0  测试集： 0.5
训练集： 1.0  测试集： 0.4
训练集： 1.0  测试集： 0.8
训练集： 1.0  测试集： 0.3
训练集： 1.0  测试集： 0.4
训练集： 1.0  测试集： 0.4444444444444444
训练集： 1.0  测试集： 0.6666666666666666
估计泛化误差：0.541


In [4]:
# SVM + pca
# 感觉要被baseline薄纱了
mydataset = myDataset(pca=1)
print(mydataset.X.shape)
record = []
for X_train, Y_train, X_test, Y_test in mydataset.DataIter():
    model = svm.SVC(
        C=1.0,
        kernel='linear',
        gamma='scale',
        decision_function_shape='ovo'
    )
    model.fit(X_train, Y_train.ravel())    #ravel函数在降维时默认是行序优先
    #利用classifier.score（）分别计算训练集和测试集的准确率。
    train_score = model.score(X_train, Y_train)
    print("训练集：", train_score,  end='  ')
    test_score = model.score(X_test, Y_test)
    print("测试集：",test_score)
    record.append(test_score)
    
print(f'估计泛化误差：{sum(record)/len(record):.3f}')


(98, 15)
训练集： 0.875  测试集： 0.5
训练集： 0.8636363636363636  测试集： 0.6
训练集： 0.8409090909090909  测试集： 0.6
训练集： 0.7954545454545454  测试集： 0.7
训练集： 0.8636363636363636  测试集： 0.5
训练集： 0.8636363636363636  测试集： 0.4
训练集： 0.8295454545454546  测试集： 0.7
训练集： 0.8068181818181818  测试集： 0.5
训练集： 0.8651685393258427  测试集： 0.4444444444444444
训练集： 0.8089887640449438  测试集： 0.7777777777777778
估计泛化误差：0.572


In [5]:
# Random Forest
from gtda.plotting import plot_point_cloud
plot_point_cloud(mydataset.X)